# None Seldon/Serve Example HuggingFace
This is the none Seldon (local implementation) of what I'm going to implement in the next section on top of Seldon core

In [6]:
from transformers import GPT2Tokenizer, TFGPT2Model
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2Model.from_pretrained('gpt2', from_pt=True)
model.save_pretrained("./tfgpt2model", saved_model=True)


text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

count = 0
max_gen_len = 10
gen_sentence = text
while count < max_gen_len:
    input_ids = tokenizer.encode(gen_sentence, return_tensors="tf")
    shape = input_ids.shape.as_list()
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# input_text = "I enjoy working in Seldon"
# count = 0
# max_gen_len = 10
# gen_sentence = input_text

# TODO start with the source notebook
# TODO find out how to translate back logits into results https://huggingface.co/course/chapter6/3?fw=pt
# TODO read the theory of paper
# TODO read this for theory https://huggingface.co/blog/how-to-generate

output

All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


INFO:tensorflow:Assets written to: ./tfgpt2model/saved_model/1/assets


INFO:tensorflow:Assets written to: ./tfgpt2model/saved_model/1/assets


TFBaseModelOutputWithPast(last_hidden_state=<tf.Tensor: shape=(1, 10, 768), dtype=float32, numpy=
array([[[ 0.16212602, -0.21588601, -0.1416967 , ..., -0.26142514,
         -0.08208874,  0.00903485],
        [ 0.46271825,  0.02478052, -0.07850052, ..., -0.08602729,
          0.5116989 , -0.39420202],
        [-0.06408656,  0.15478222, -0.6306318 , ...,  0.24884915,
          0.36889777,  0.08327568],
        ...,
        [-0.5588062 , -0.44919217, -1.4540867 , ...,  0.16513371,
         -0.13031706, -0.37356085],
        [ 0.13995807, -0.38752714, -0.79088706, ..., -0.17763259,
          0.18202344,  0.21815276],
        [ 0.17187777, -0.24292202, -0.11242812, ..., -0.10721997,
          0.12059753, -0.3214079 ]]], dtype=float32)>, past_key_values=(<tf.Tensor: shape=(2, 1, 12, 10, 64), dtype=float32, numpy=
array([[[[[-1.07186723e+00,  2.41698933e+00,  9.66034114e-01, ...,
           -4.78705704e-01, -3.31556886e-01,  1.79252303e+00],
          [-2.28969359e+00,  2.54237008e+00,  8.317

In [ ]:
import numpy as np

while count < max_gen_len:
    input_ids = tokenizer.encode(gen_sentence, return_tensors="tf")
    shape = input_ids.shape.as_list()
    payload = {
        "inputs": [
            {
                "name": "input_ids",
                "datatype": "INT32",
                "shape": shape,
                "data": input_ids.numpy().tolist(),
            },
            {
                "name": "attention_mask",
                "datatype": "INT32",
                "shape": shape,
                "data": np.ones(shape, dtype=np.int32).tolist(),
            },
        ]
    }

    ret = requests.post(
        "http://localhost:8004/seldon/seldon/gpt2/v2/models/gpt2/infer", json=payload
    )

    try:
        res = ret.json()
    except:
        continue

    # extract logits
    logits = np.array(res["outputs"][1]["data"])
    logits = logits.reshape(res["outputs"][1]["shape"])

    # take the best next token probability of the last token of input ( greedy approach)
    next_token = logits.argmax(axis=2)[0]
    next_token_str = tokenizer.decode(
        next_token[-1:], skip_special_tokens=True, clean_up_tokenization_spaces=True
    ).strip()
    gen_sentence += " " + next_token_str
    count += 1

print(f"Input: {input_text}\nOutput: {gen_sentence}")

# Server Examples

## Triton Server+TensorFlow+Seldon

## Triton Server+Torch+Seldon

## Triton Server

## TfServing+Seldon

## TfServing

## TorchServe

## Kserve

# Load Testing